In [1]:
import json
import warnings
from typing import Any, Literal

import numpy as np
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)


# Demo (Prevents ruff from removing the unused module import)
name: Any
category: Literal["A", "B", "C"]
json.loads('{"name": "Bike Rental Prediction", "category": "A"}')

{'name': 'Bike Rental Prediction', 'category': 'A'}

In [3]:
go_up_from_current_directory(go_up=1)

/Users/mac/Desktop/Projects/Bike-Rental-Prediction


In [12]:
from src.config import app_config
from src.exceptions import (
    CustomError,
    MLFlowConnectionError,
)
from src.ml.feature_engineering import FeatureEngineer
from src.ml.trainer import ModelTrainer
from src.ml.utils import split_temporal_data_to_train_val_test

In [8]:
import httpx
import mlflow
import mlflow.sklearn

port: int = 5001
url: str = f"http://localhost:{port}"


def end_mlflow_run() -> None:
    """Ends the current MLflow run if one is active."""
    try:
        mlflow.end_run()
        console.print("MLflow run ended successfully.")
    except Exception as e:
        console.print(e)


def check_mlflow(url: str, timeout: float = 2.0) -> bool:
    """
    Check MLflow endpoint accessibility and handle common httpx errors.

    Returns True if reachable (2xx), False otherwise.
    """
    try:
        resp = httpx.get(url, timeout=timeout)
        resp.raise_for_status()
        console.print("[success]MLflow is accessible")
        return True

    except httpx.HTTPStatusError as e:
        console.print(
            f"[error]MLflow returned non-2xx status: {e.response.status_code} — {e}"
        )
        return False

    except httpx.RequestError as e:
        # covers ConnectError, ReadTimeout, etc.
        console.print(f"[error]Network/connection error when contacting MLflow: {e}")
        return False

    except (MLFlowConnectionError, CustomError) as e:
        console.print(f"[error]Project-specific MLflow error: {e}")
        return False

    except Exception as e:
        console.print(f"[error]Unexpected error: {e}")
        return False


check_mlflow(url)

MLflow is accessible

True

In [9]:
fp: str = "../../../../Documents/data_dump/bike_data/database.parquet"
data: pl.DataFrame = pl.read_parquet(fp)
console.print(f"Shape: {data.shape}", style="info")
display(data.head())

(train_df, val_df, test_df) = split_temporal_data_to_train_val_test(data=data)

Shape: (13903, 16)

datetime,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
str,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64
"""2011-01-01 00:00:00""",1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
"""2011-01-01 01:00:00""",1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0.0,8,32,40
"""2011-01-01 02:00:00""",1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0.0,5,27,32
"""2011-01-01 03:00:00""",1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
"""2011-01-01 04:00:00""",1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


Shapes -> Train shape: (8342, 16) | Val shape: (2780, 16) | Test shape: (2781, 16)


In [10]:
train_df.sample(2).to_dicts()

# pl.from_records(train_df.sample().to_dicts())

[{'datetime': '2011-04-03 12:00:00',
  'season': 2,
  'yr': 0,
  'mnth': 4,
  'hr': 12,
  'holiday': 0,
  'weekday': 0,
  'workingday': 0,
  'weathersit': 1,
  'temp': 0.44,
  'atemp': 0.4394,
  'hum': 0.38,
  'windspeed': 0.2836,
  'casual': 166,
  'registered': 147,
  'cnt': 313},
 {'datetime': '2011-03-02 15:00:00',
  'season': 1,
  'yr': 0,
  'mnth': 3,
  'hr': 15,
  'holiday': 0,
  'weekday': 3,
  'workingday': 1,
  'weathersit': 1,
  'temp': 0.54,
  'atemp': 0.5152,
  'hum': 0.19,
  'windspeed': 0.4179,
  'casual': 19,
  'registered': 91,
  'cnt': 110}]

In [13]:
feat_eng = FeatureEngineer()
train_features_df: pl.DataFrame = feat_eng.transform(
    data=train_df, config=app_config.feature_config
)
val_features_df: pl.DataFrame = feat_eng.transform(
    data=val_df, config=app_config.feature_config
)
test_features_df: pl.DataFrame = feat_eng.transform(
    data=test_df, config=app_config.feature_config
)
display(train_features_df.head())

val_features_df.head()

season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,is_weekend,sin_hour,cos_hour,sin_weekday,cos_weekday,cnt_lag_0hr,cnt_lag_1hr,cnt_lag_24hr,hr_lag_1hr,hr_lag_24hr,temp_lag_1hr,temp_lag_3hr,cnt_rolling_mean_3hr,cnt_rolling_median_3hr,cnt_rolling_mean_6hr,cnt_rolling_median_6hr,hr_rolling_mean_3hr,hr_rolling_median_3hr,hr_rolling_mean_6hr,hr_rolling_median_6hr,temp_plus_hum,hum_plus_hr,cnt_diff_1hr,cnt_diff_2hr,hr_diff_1hr,hr_diff_24hr,temp_diff_1hr,temp_diff_2hr,temp_diff_24hr,is_high_temp,is_high_hum,is_peak_hour,is_working_hour,is_business_hour,target
i64,i64,i64,i64,i64,i64,i64,f64,f64,i8,f64,f64,f64,f64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64,i8,i8,i8,i8,i8,i64
1,1,0,0,6,0,1,0.24,0.81,1,0.0,1.0,-0.781831,0.62349,16,16,16,0,0,0.24,0.24,29.333333,32.0,17.166667,14.5,1.0,1.0,2.5,2.5,1.05,0.81,24,16,1,0,-0.02,-0.02,0.22,0,0,0,0,0,40
1,1,1,0,6,0,1,0.22,0.8,1,0.258819,0.965926,-0.781831,0.62349,40,16,16,0,0,0.24,0.24,29.333333,32.0,17.166667,14.5,1.0,1.0,2.5,2.5,1.02,1.8,24,16,1,0,-0.02,-0.02,0.22,0,0,0,0,0,32
1,1,2,0,6,0,1,0.22,0.8,1,0.5,0.866025,-0.781831,0.62349,32,40,16,1,0,0.22,0.24,29.333333,32.0,17.166667,14.5,1.0,1.0,2.5,2.5,1.02,2.8,-8,16,1,0,0.0,-0.02,0.22,0,0,0,0,0,13
1,1,3,0,6,0,1,0.24,0.75,1,0.707107,0.707107,-0.781831,0.62349,13,32,16,2,0,0.22,0.24,28.333333,32.0,17.166667,14.5,2.0,2.0,2.5,2.5,0.99,3.75,-19,-27,1,0,0.02,0.02,0.22,0,0,0,0,0,1
1,1,4,0,6,0,1,0.24,0.75,1,0.866025,0.5,-0.781831,0.62349,1,13,16,3,0,0.24,0.22,15.333333,13.0,17.166667,14.5,3.0,3.0,2.5,2.5,0.99,4.75,-12,-31,1,0,0.0,0.02,0.22,0,0,0,0,0,1


season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,is_weekend,sin_hour,cos_hour,sin_weekday,cos_weekday,cnt_lag_0hr,cnt_lag_1hr,cnt_lag_24hr,hr_lag_1hr,hr_lag_24hr,temp_lag_1hr,temp_lag_3hr,cnt_rolling_mean_3hr,cnt_rolling_median_3hr,cnt_rolling_mean_6hr,cnt_rolling_median_6hr,hr_rolling_mean_3hr,hr_rolling_median_3hr,hr_rolling_mean_6hr,hr_rolling_median_6hr,temp_plus_hum,hum_plus_hr,cnt_diff_1hr,cnt_diff_2hr,hr_diff_1hr,hr_diff_24hr,temp_diff_1hr,temp_diff_2hr,temp_diff_24hr,is_high_temp,is_high_hum,is_peak_hour,is_working_hour,is_business_hour,target
i64,i64,i64,i64,i64,i64,i64,f64,f64,i8,f64,f64,f64,f64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64,i8,i8,i8,i8,i8,i64
4,12,6,0,1,1,1,0.16,0.86,0,1.0,6.1232e-17,0.781831,0.62349,68,68,68,6,6,0.16,0.16,218.666667,191.0,179.0,155.0,7.0,7.0,8.5,8.5,1.02,6.86,123,329,1,0,0.02,0.04,0.2,0,1,0,0,0,191
4,12,7,0,1,1,1,0.18,0.74,0,0.965926,-0.258819,0.781831,0.62349,191,68,68,6,6,0.16,0.16,218.666667,191.0,179.0,155.0,7.0,7.0,8.5,8.5,0.92,7.74,123,329,1,0,0.02,0.04,0.2,0,0,1,0,0,397
4,12,8,0,1,1,1,0.2,0.75,0,0.866025,-0.5,0.781831,0.62349,397,191,68,7,6,0.18,0.16,218.666667,191.0,179.0,155.0,7.0,7.0,8.5,8.5,0.95,8.75,206,329,1,0,0.02,0.04,0.2,0,0,1,0,1,183
4,12,9,0,1,1,1,0.22,0.69,0,0.707107,-0.707107,0.781831,0.62349,183,397,68,8,6,0.2,0.16,257.0,191.0,179.0,155.0,8.0,8.0,8.5,8.5,0.91,9.69,-214,-8,1,0,0.02,0.04,0.2,0,0,1,1,1,108
4,12,10,0,1,1,1,0.24,0.67,0,0.5,-0.866025,0.781831,0.62349,108,183,68,9,6,0.22,0.18,229.333333,183.0,179.0,155.0,9.0,9.0,8.5,8.5,0.91,10.67,-75,-289,1,0,0.02,0.04,0.2,0,0,0,1,1,127


In [14]:
trainer = ModelTrainer(
    train_data=train_features_df,
    val_data=val_features_df,
    test_data=test_features_df,
    target_col="target",
)

2025-10-17 18:41:52 - trainer - [INFO] - Data prepared -> x_train shape: (8342, 43), y_train shape: (8342,) | x_val shape: (2780, 43), y_val shape: (2780,) | x_test shape: (2781, 43), y_test shape: (2781,)
2025-10-17 18:41:52 - mlflow_tracker - [INFO] - Set MLflow tracking URI to: http://localhost:5001
2025-10-17 18:41:52 - mlflow_tracker - [INFO] - Created new experiment: bike rental (ID: 1)
2025-10-17 18:41:52 - mlflow_tracker - [INFO] - Set experiment to: bike rental (ID: 1)
2025-10-17 18:41:52 - mlflow_tracker - [INFO] - Initialized MLFlowTracker with experiment: bike rental


In [15]:
# trainer._hyperparameter_tuning_xgboost()
# result = trainer._hyperparameter_tuning_lightgbm()

# trainer.hyperparameter_tuning_all_models()
result = trainer.train_all_models()

2025-10-17 18:42:05 - trainer - [INFO] - 🚀 Training with default hyperparameters
2025-10-17 18:42:05 - mlflow_tracker - [INFO] - Started MLflow run: 1cc184d507bc41fb96147cf6cc320767 (name: run_2025-10-17T18:42:05)
2025-10-17 18:42:05 - trainer - [INFO] - Training Random Forest ...
2025-10-17 18:42:05 - mlflow_tracker - [INFO] - Started MLflow run: eb0b50b08ef54d46b47b478a8bb870ae (name: run_2025-10-17T18:42:05)
2025-10-17 18:42:05 - trainer - [INFO] - Starting Random Forest training with TimeSeriesSplit cross-validation.
2025-10-17 18:42:26 - mlflow_tracker - [INFO] - ✅ Successfully logged ModelType.RANDOM_FOREST model and metadata
🏃 View run run_2025-10-17T18:42:05 at: http://localhost:5001/#/experiments/1/runs/eb0b50b08ef54d46b47b478a8bb870ae
🧪 View experiment at: http://localhost:5001/#/experiments/1
2025-10-17 18:42:26 - mlflow_tracker - [INFO] - Ended MLflow run with status: FINISHED


2025-10-17 18:42:26 - mlflow_s3_utils - [INFO] - Synced models/feat_imp_random_forest.json to S3
2025-10-17 18:42:26 - mlflow_s3_utils - [INFO] - Synced models/ModelType.RANDOM_FOREST/ModelType.RANDOM_FOREST_2025-10-17T18:42:26_metadata.yaml to S3
2025-10-17 18:42:26 - mlflow_s3_utils - [INFO] - Synced models/ModelType.RANDOM_FOREST/ModelType.RANDOM_FOREST_2025-10-17T18:42:26_input_example.json to S3
2025-10-17 18:42:27 - mlflow_s3_utils - [INFO] - Synced models/ModelType.RANDOM_FOREST/ModelType.RANDOM_FOREST_2025-10-17T18:42:26_model.pkl to S3
2025-10-17 18:42:27 - mlflow_s3_utils - [INFO] - Successfully synced all artifacts for run eb0b50b08ef54d46b47b478a8bb870ae to S3
2025-10-17 18:42:27 - mlflow_tracker - [INFO] - ✅ Synced artifacts to S3 for run eb0b50b08ef54d46b47b478a8bb870ae
2025-10-17 18:42:27 - trainer - [INFO] - 🚀 Random Forest training completed successfully.
2025-10-17 18:42:27 - trainer - [INFO] - Training XGBoost ...
2025-10-17 18:42:27 - mlflow_tracker - [INFO] - Start

2025-10-17 18:42:27 - mlflow_s3_utils - [INFO] - Synced models/feat_imp_xgboost.json to S3
2025-10-17 18:42:27 - mlflow_s3_utils - [INFO] - Synced models/ModelType.XGBOOST/ModelType.XGBOOST_2025-10-17T18:42:27_metadata.yaml to S3
2025-10-17 18:42:27 - mlflow_s3_utils - [INFO] - Synced models/ModelType.XGBOOST/ModelType.XGBOOST_2025-10-17T18:42:27_input_example.json to S3
2025-10-17 18:42:27 - mlflow_s3_utils - [INFO] - Synced models/ModelType.XGBOOST/ModelType.XGBOOST_2025-10-17T18:42:27_model.json to S3
2025-10-17 18:42:27 - mlflow_s3_utils - [INFO] - Successfully synced all artifacts for run f2b7243b004f429b8ff7a9f716e4e226 to S3
2025-10-17 18:42:27 - mlflow_tracker - [INFO] - ✅ Synced artifacts to S3 for run f2b7243b004f429b8ff7a9f716e4e226
2025-10-17 18:42:27 - trainer - [INFO] - 🚀 XGBoost training completed successfully.
2025-10-17 18:42:27 - trainer - [INFO] - Training LightGBM ...
2025-10-17 18:42:27 - mlflow_tracker - [INFO] - Started MLflow run: e73a599cf1b74a04b151d6b45d7cfe4

2025-10-17 18:42:29 - mlflow_s3_utils - [INFO] - Synced models/feat_imp_lightgbm.json to S3
2025-10-17 18:42:29 - mlflow_s3_utils - [INFO] - Synced models/ModelType.LIGHTGBM/ModelType.LIGHTGBM_2025-10-17T18:42:28_model.txt to S3
2025-10-17 18:42:29 - mlflow_s3_utils - [INFO] - Synced models/ModelType.LIGHTGBM/ModelType.LIGHTGBM_2025-10-17T18:42:28_input_example.json to S3
2025-10-17 18:42:29 - mlflow_s3_utils - [INFO] - Synced models/ModelType.LIGHTGBM/ModelType.LIGHTGBM_2025-10-17T18:42:28_metadata.yaml to S3
2025-10-17 18:42:29 - mlflow_s3_utils - [INFO] - Successfully synced all artifacts for run e73a599cf1b74a04b151d6b45d7cfe4f to S3
2025-10-17 18:42:29 - mlflow_tracker - [INFO] - ✅ Synced artifacts to S3 for run e73a599cf1b74a04b151d6b45d7cfe4f
2025-10-17 18:42:29 - trainer - [INFO] - 🚀 LightGBM training completed successfully.
2025-10-17 18:42:29 - trainer - [INFO] - ✅ ALL models training completed successfully.
2025-10-17 18:42:29 - visualization - [INFO] - Saved grouped metrics

2025-10-17 18:42:30 - mlflow_s3_utils - [INFO] - Synced visualizations/reports/model_metrics_comparison_2025-10-17T18:42:29.html to S3
2025-10-17 18:42:30 - mlflow_s3_utils - [INFO] - Successfully synced all artifacts for run 1cc184d507bc41fb96147cf6cc320767 to S3
2025-10-17 18:42:30 - mlflow_tracker - [INFO] - ✅ Synced artifacts to S3 for run 1cc184d507bc41fb96147cf6cc320767
2025-10-17 18:42:30 - mlflow_tracker - [INFO] - Ended MLflow run with status: FINISHED
2025-10-17 18:42:30 - trainer - [INFO] - Syncing artifacts to S3...


2025-10-17 18:42:30 - mlflow_s3_utils - [INFO] - Synced visualizations/reports/model_metrics_comparison_2025-10-17T18:42:29.html to S3
2025-10-17 18:42:30 - mlflow_s3_utils - [INFO] - Successfully synced all artifacts for run 1cc184d507bc41fb96147cf6cc320767 to S3
2025-10-17 18:42:30 - trainer - [INFO] - ✅ Successfully synced artifacts to S3
2025-10-17 18:42:30 - trainer - [INFO] - Verifying S3 artifact storage...
2025-10-17 18:42:30 - s3_verification - [INFO] - Found 1 artifacts in S3 for run 1cc184d507bc41fb96147cf6cc320767
2025-10-17 18:42:30 - s3_verification - [INFO] - Artifacts: visualizations/reports/model_metrics_comparison_2025-10-17T18:42:29.html...
2025-10-17 18:42:30 - s3_verification - [INFO] - ✅ S3 artifact verification PASSED
2025-10-17 18:42:30 - s3_verification - [INFO] -   - Artifact URI: s3://mlflow-artifacts/1/1cc184d507bc41fb96147cf6cc320767/artifacts
2025-10-17 18:42:30 - s3_verification - [INFO] -   - Total artifacts: 1


In [16]:
trainer.mlflow_tracker.end_run()

2025-10-17 18:46:15 - mlflow_tracker - [INFO] - Ended MLflow run with status: FINISHED


In [ ]:
# 25
col: str = "windspeed"
train_df[col].describe().to_dicts()

In [ ]:
{
    "data_shape": {
        "total_rows": 13903,
        "total_columns": 16,
        "number_of_numeric_columns": 15,
        "number_of_categorical_columns": 1,
    },
    "other_info": {
        "data_nulls": {
            "hr": 0,
            "yr": 0,
            "cnt": 0,
            "hum": 0,
            "mnth": 0,
            "temp": 0,
            "atemp": 0,
            "casual": 0,
            "season": 0,
            "holiday": 0,
            "weekday": 0,
            "datetime": 0,
            "windspeed": 0,
            "registered": 0,
            "weathersit": 0,
            "workingday": 0,
        },
        "total_nulls": 0,
        "memory_usage_MB": 1.84,
        "num_duplicated_rows": 0,
        "validation_timestamp": "2025-10-14T12:58:00+00:00",
        "num_unique_numeric_rows": {
            "hr": 24,
            "yr": 2,
            "cnt": 789,
            "hum": 88,
            "mnth": 12,
            "temp": 50,
            "atemp": 65,
            "casual": 303,
            "season": 4,
            "holiday": 2,
            "weekday": 7,
            "windspeed": 30,
            "registered": 701,
            "weathersit": 4,
            "workingday": 2,
        },
        "num_unique_categorical_rows": {"datetime": 13903},
    },
    "data_schema": {
        "numeric": {
            "hr": "Int64",
            "yr": "Int64",
            "cnt": "Int64",
            "hum": "Float64",
            "mnth": "Int64",
            "temp": "Float64",
            "atemp": "Float64",
            "casual": "Int64",
            "season": "Int64",
            "holiday": "Int64",
            "weekday": "Int64",
            "windspeed": "Float64",
            "registered": "Int64",
            "weathersit": "Int64",
            "workingday": "Int64",
        },
        "categorical": {},
    },
    "summary_statistics": {
        "numeric": [
            {
                "max": 4.0,
                "min": 1.0,
                "std": 1.04,
                "mean": 2.26,
                "mode": [2.0],
                "count": 13903,
                "range": 3.0,
                "column": "season",
                "median": 2.0,
                "variance": 1.07,
                "missing_pct": 0.0,
                "unique_values": 4,
                "missing_values": 0,
            },
            {
                "max": 1.0,
                "min": 0.0,
                "std": 0.48,
                "mean": 0.38,
                "mode": [0.0],
                "count": 13903,
                "range": 1.0,
                "column": "yr",
                "median": 0.0,
                "variance": 0.24,
                "missing_pct": 0.0,
                "unique_values": 2,
                "missing_values": 0,
            },
            {
                "max": 12.0,
                "min": 1.0,
                "std": 3.22,
                "mean": 5.65,
                "mode": [7.0, 5.0],
                "count": 13903,
                "range": 11.0,
                "column": "mnth",
                "median": 5.0,
                "variance": 10.36,
                "missing_pct": 0.0,
                "unique_values": 12,
                "missing_values": 0,
            },
            {
                "max": 23.0,
                "min": 0.0,
                "std": 6.91,
                "mean": 11.55,
                "mode": [16.0, 17.0],
                "count": 13903,
                "range": 23.0,
                "column": "hr",
                "median": 12.0,
                "variance": 47.78,
                "missing_pct": 0.0,
                "unique_values": 24,
                "missing_values": 0,
            },
            {
                "max": 1.0,
                "min": 0.0,
                "std": 0.16,
                "mean": 0.03,
                "mode": [0.0],
                "count": 13903,
                "range": 1.0,
                "column": "holiday",
                "median": 0.0,
                "variance": 0.03,
                "missing_pct": 0.0,
                "unique_values": 2,
                "missing_values": 0,
            },
            {
                "max": 6.0,
                "min": 0.0,
                "std": 2.01,
                "mean": 3.0,
                "mode": [6.0],
                "count": 13903,
                "range": 6.0,
                "column": "weekday",
                "median": 3.0,
                "variance": 4.02,
                "missing_pct": 0.0,
                "unique_values": 7,
                "missing_values": 0,
            },
            {
                "max": 1.0,
                "min": 0.0,
                "std": 0.46,
                "mean": 0.68,
                "mode": [1.0],
                "count": 13903,
                "range": 1.0,
                "column": "workingday",
                "median": 1.0,
                "variance": 0.22,
                "missing_pct": 0.0,
                "unique_values": 2,
                "missing_values": 0,
            },
            {
                "max": 4.0,
                "min": 1.0,
                "std": 0.64,
                "mean": 1.42,
                "mode": [1.0],
                "count": 13903,
                "range": 3.0,
                "column": "weathersit",
                "median": 1.0,
                "variance": 0.41,
                "missing_pct": 0.0,
                "unique_values": 4,
                "missing_values": 0,
            },
            {
                "max": 1.0,
                "min": 0.02,
                "std": 0.2,
                "mean": 0.5,
                "mode": [0.62],
                "count": 13903,
                "range": 0.98,
                "column": "temp",
                "median": 0.5,
                "variance": 0.04,
                "missing_pct": 0.0,
                "unique_values": 50,
                "missing_values": 0,
            },
            {
                "max": 1.0,
                "min": 0.0,
                "std": 0.18,
                "mean": 0.48,
                "mode": [0.62],
                "count": 13903,
                "range": 1.0,
                "column": "atemp",
                "median": 0.48,
                "variance": 0.03,
                "missing_pct": 0.0,
                "unique_values": 65,
                "missing_values": 0,
            },
            {
                "max": 1.0,
                "min": 0.0,
                "std": 0.2,
                "mean": 0.62,
                "mode": [0.88],
                "count": 13903,
                "range": 1.0,
                "column": "hum",
                "median": 0.62,
                "variance": 0.04,
                "missing_pct": 0.0,
                "unique_values": 88,
                "missing_values": 0,
            },
            {
                "max": 0.85,
                "min": 0.0,
                "std": 0.12,
                "mean": 0.19,
                "mode": [0.0],
                "count": 13903,
                "range": 0.85,
                "column": "windspeed",
                "median": 0.19,
                "variance": 0.02,
                "missing_pct": 0.0,
                "unique_values": 30,
                "missing_values": 0,
            },
            {
                "max": 367.0,
                "min": 0.0,
                "std": 47.34,
                "mean": 34.04,
                "mode": [0.0],
                "count": 13903,
                "range": 367.0,
                "column": "casual",
                "median": 15.0,
                "variance": 2241.19,
                "missing_pct": 0.0,
                "unique_values": 303,
                "missing_values": 0,
            },
            {
                "max": 796.0,
                "min": 0.0,
                "std": 137.51,
                "mean": 140.6,
                "mode": [4.0],
                "count": 13903,
                "range": 796.0,
                "column": "registered",
                "median": 107.0,
                "variance": 18909.65,
                "missing_pct": 0.0,
                "unique_values": 701,
                "missing_values": 0,
            },
            {
                "max": 957.0,
                "min": 1.0,
                "std": 166.96,
                "mean": 174.64,
                "mode": [5.0],
                "count": 13903,
                "range": 956.0,
                "column": "cnt",
                "median": 130.0,
                "variance": 27874.11,
                "missing_pct": 0.0,
                "unique_values": 789,
                "missing_values": 0,
            },
        ],
        "categorical": [
            {
                "column": "datetime",
                "missing_pct": 0.0,
                "total_count": 13903,
                "value_counts": [
                    {
                        "__data__": ["2011-01-01 00:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                    {
                        "__data__": ["2011-01-01 01:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                    {
                        "__data__": ["2011-01-01 02:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                    {
                        "__data__": ["2011-01-01 03:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                    {
                        "__data__": ["2011-01-01 04:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                    {
                        "__data__": ["2011-01-01 05:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                    {
                        "__data__": ["2011-01-01 06:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                    {
                        "__data__": ["2011-01-01 07:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                    {
                        "__data__": ["2011-01-01 08:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                    {
                        "__data__": ["2011-01-01 09:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                ],
                "unique_values": 13903,
                "missing_values": 0,
            }
        ],
    },
}

𝗧𝗵𝗲 𝗙𝘂𝗹𝗹 𝗦𝘁𝗮𝗰𝗸 𝟳-𝗦𝘁𝗲𝗽𝘀 𝗠𝗟𝗢𝗽𝘀 𝗙𝗿𝗮𝗺𝗲𝘄𝗼𝗿𝗸
